In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
dfpathL = glob('data/gs1029/*')
dfL = []
for p in dfpathL[:100]:
  df_ = pd.read_csv(p)
  dfL.append(df_)
datadf = pd.concat(dfL).drop(columns='Unnamed: 0',)
# datadf

In [3]:
datadf

,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,cond,trial,acc
0,88.040869,75.020201,60.275032,1.083564,0,1,1,1,blocked,0,0.5
1,88.040869,75.020201,60.275032,1.083564,0,1,1,1,blocked,1,0.5
2,88.040869,75.020201,60.275032,1.083564,0,1,1,1,blocked,2,0.5
3,88.040869,75.020201,60.275032,1.083564,0,1,1,1,blocked,3,0.5
4,88.040869,75.020201,60.275032,1.083564,0,1,1,1,blocked,4,0.5
...,...,...,...,...,...,...,...,...,...,...,...
9995,83.189760,14.752879,23.854439,0.504802,0,1,1,1,late,195,0.5
9996,83.189760,14.752879,23.854439,0.504802,0,1,1,1,late,196,0.5
9997,83.189760,14.752879,23.854439,0.504802,0,1,1,1,late,197,0.5
9998,83.189760,14.752879,23.854439,0.504802,0,1,1,1,late,198,0.5


In [4]:
datadf.columns
paramL = ['concentration', 'stickiness_wi', 'stickiness_bt',
       'sparsity', 'pvar', 'lrate', 'lratep', 'decay_rate']

### compute df metrics, form df with summary results

In [5]:
## compute df
groupvars = paramL 
gsdf_group = datadf.groupby(groupvars)
dfL = []
for params_i,df_i in gsdf_group:
  dataD = {**dict(zip(groupvars,params_i))}
  ## loop conditions (BIEML)
  for cond_i,df_c in df_i.groupby('cond'):
    ## compute metrics
    testacc = np.mean(df_c.acc[-40:])
    ## populate dataD
    dataD['testacc-%s'%cond_i[0]] = testacc
  # 
  dfL.append(pd.DataFrame(index=[0],data=dataD))
gsdf = pd.concat(dfL)
gsdf

,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,testacc-b,testacc-e,testacc-i,testacc-l,testacc-m
0,0.004664,58.717716,59.266231,0.047673,0,1,1,1,0.499982,0.498991,0.499859,0.499957,0.500681
0,0.030341,98.216755,64.134380,0.923695,0,1,1,1,0.500606,0.500572,0.500221,0.500675,0.499857
0,0.076489,80.638197,85.506653,0.198552,0,1,1,1,0.499724,0.498898,0.499154,0.499355,0.500070
0,0.117835,40.185496,79.416095,0.104825,0,1,1,1,0.501409,0.497825,0.501230,0.501036,0.501142
0,0.180072,98.576930,81.183617,1.119720,0,1,1,1,0.501929,0.499447,0.499336,0.498950,0.501193
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,99.931811,47.814880,20.548172,0.969261,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000
0,99.934152,58.491512,4.804470,0.811958,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000
0,99.964592,5.719831,82.993313,0.966900,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000
0,99.968973,38.146926,51.773209,0.447346,0,1,1,1,0.500000,0.500000,0.500000,0.500000,0.500000


In [6]:
Btest = gsdf.loc[:,'testacc-b'] > 0.8
Itest1 = gsdf.loc[:,'testacc-i'] > 0.5
Itest2 = gsdf.loc[:,'testacc-i'] < 0.7
Etest = gsdf.loc[:,'testacc-e'] > 0.8
Mtest = gsdf.loc[:,'testacc-m'] > 0.8
Ltest = gsdf.loc[:,'testacc-l'] > 0.8
MItest = gsdf.loc[:,'testacc-m'] - gsdf.loc[:,'testacc-i'] > 0.1
LItest = gsdf.loc[:,'testacc-l'] - gsdf.loc[:,'testacc-i'] > 0.1

C = Btest&Itest2&Itest1&MItest
sgsdf = gsdf[C]
sgsdf

,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,testacc-b,testacc-e,testacc-i,testacc-l,testacc-m
0,21.485931,93.964103,14.390017,0.578377,0,1,1,1,0.825279,0.815403,0.694483,0.824373,0.815218
0,23.330337,84.675328,32.144767,0.505522,0,1,1,1,0.819130,0.823311,0.686737,0.815256,0.842585
0,24.098843,86.592935,43.879893,0.260369,0,1,1,1,0.806853,0.804285,0.656813,0.803087,0.793973
0,24.259225,85.017125,39.269949,0.288137,0,1,1,1,0.818456,0.799957,0.677825,0.804385,0.830257
0,25.034719,70.895431,40.016575,0.387799,0,1,1,1,0.809747,0.815341,0.673426,0.819814,0.830230
0,25.479305,93.392889,34.491882,0.512057,0,1,1,1,0.828699,0.824431,0.687676,0.807823,0.803202
0,26.007892,91.818111,39.724270,0.247009,0,1,1,1,0.822887,0.805118,0.671642,0.802923,0.807257
0,27.456647,84.216111,26.500180,0.226853,0,1,1,1,0.819131,0.810752,0.682759,0.825513,0.793145
0,29.830020,74.866509,35.120705,0.405619,0,1,1,1,0.816255,0.781785,0.652958,0.789377,0.797624
0,30.139553,99.800139,46.551902,0.311592,0,1,1,1,0.811089,0.788327,0.659979,0.791198,0.791800


In [31]:
df = datadf.set_index(paramL)

In [32]:
paramL

['concentration',
 'stickiness_wi',
 'stickiness_bt',
 'sparsity',
 'pvar',
 'lrate',
 'lratep',
 'decay_rate']

In [38]:
df.loc[df.index[54]]

<ipython-input-38-85b4ee111eab>:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[df.index[54]]


cond  \
concentration stickiness_wi stickiness_bt sparsity pvar lrate lratep decay_rate            
88.040869     75.020201     60.275032     1.083564 0    1     1      1           blocked   
                                                                     1           blocked   
                                                                     1           blocked   
                                                                     1           blocked   
                                                                     1           blocked   
...                                                                                  ...   
                                                                     1              late   
                                                                     1              late   
                                                                     1              late   
                                                                     1              late   
                                                                     1              late   

                                                                                 trial  \
concentration stickiness_wi stickiness_bt sparsity pvar lrate lratep decay_rate          
88.040869     75.020201     60.275032     1.083564 0    1     1      1               0   
                                                                     1               1   
                                                                     1               2   
                                                                     1               3   
                                                                     1               4   
...                                                                                ...   
                                                                     1             195   
                                                                     1             196   
                                                                     1             197   
                                                                     1             198   
                                                                     1             199   

                                                                                 acc  
concentration stickiness_wi stickiness_bt sparsity pvar lrate lratep decay_rate       
88.040869     75.020201     60.275032     1.083564 0    1     1      1           0.5  
                                                                     1           0.5  
                                                                     1           0.5  
                                                                     1           0.5  
                                                                     1           0.5  
...                                                                              ...  
                                                                     1           0.5  
                                                                     1           0.5  
                                                                     1           0.5  
                                                                     1           0.5  
                                                                     1           0.5  

[1000 rows x 3 columns]

In [11]:
pvals = (30.630502,89.728344,37.812341,0.666619,0,1,1)
datadf.loc[:,pvals]

KeyError: "None of [Float64Index([30.630502, 89.728344, 37.812341, 0.666619, 0.0, 1.0, 1.0], dtype='float64')] are in the [columns]"

### plot single params

In [7]:

paramL
np.logical_and(*[datadf.loc[:,p] == i for p,i in zip(paramL,pvals)])
# np.logical_and?

TypeError: ufunc() takes from 2 to 3arguments but 7 were given

In [ ]:
datadf.groupby(paramL)

In [ ]:
datadf.groupby(paramL).flatten().loc[:,pvals]

In [ ]:
for i,d in datadf.groupby(paramL):
  i
d
i


In [ ]:
datadf

In [ ]:
datadf.drop?

In [ ]:
datadf.groupby(paramL).get_group

In [ ]:
datadf.groupby(paramL).get_group